In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
pd.set_option('display.max_columns',100)

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [84]:
from tensorflow.keras import regularizers

In [6]:
df = pd.read_csv("Train_Data.csv")

In [7]:
df.head()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,rootshell,suattempted,numroot,numfilecreations,numshells,numaccessfiles,numoutboundcmds,ishostlogin,isguestlogin,count,srvcount,serrorrate,srvserrorrate,rerrorrate,srvrerrorrate,samesrvrate,diffsrvrate,srvdiffhostrate,dsthostcount,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack
0,0,tcp,netbios_dgm,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,277,15,0.0,0.0,1.0,1.0,0.05,0.06,0.00,255,15,0.06,0.06,0.00,0.00,0.00,0.0,1.00,1.0,21,1
1,0,tcp,smtp,SF,1239,400,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,116,0.45,0.04,0.00,0.00,0.11,0.0,0.02,0.0,18,0
2,0,tcp,http,SF,222,945,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,25,0.0,0.0,0.0,0.0,1.00,0.00,0.08,57,255,1.00,0.00,0.02,0.03,0.00,0.0,0.00,0.0,21,0
3,0,tcp,http,SF,235,1380,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,21,0
4,0,tcp,uucp_path,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,256,3,0.0,0.0,1.0,1.0,0.01,0.06,0.00,255,3,0.01,0.08,0.00,0.00,0.00,0.0,1.00,1.0,19,1


In [8]:
df.shape

(86845, 43)

In [28]:
for x in df.columns:
    print((df[x]).dtype)

int64
object
object
object
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
float64
float64
float64
float64
float64
float64
float64
int64
int64
float64
float64
float64
float64
float64
float64
float64
float64
int64
int64


In [54]:
df_removed_obj = df
l = []
for x in df.columns:
    if df[x].dtype==object:
        l.append(x)
        df_removed_obj = pd.concat([df_removed_obj, pd.get_dummies(df[x])],axis = 'columns')
df_removed_obj.drop(l,axis = 1,inplace = True)
df_removed_obj.head()

,duration,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,rootshell,suattempted,numroot,numfilecreations,numshells,numaccessfiles,numoutboundcmds,ishostlogin,isguestlogin,count,srvcount,serrorrate,srvserrorrate,rerrorrate,srvrerrorrate,samesrvrate,diffsrvrate,srvdiffhostrate,dsthostcount,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack,icmp,tcp,udp,IRC,X11,Z39_50,auth,bgp,courier,csnet_ns,...,iso_tsap,klogin,kshell,ldap,link,login,mtp,name,netbios_dgm,netbios_ns,netbios_ssn,netstat,nnsp,nntp,ntp_u,other,pop_2,pop_3,printer,private,red_i,remote_job,rje,shell,smtp,sql_net,ssh,sunrpc,supdup,systat,telnet,tftp_u,tim_i,time,urh_i,urp_i,uucp,uucp_path,vmnet,whois,OTH,REJ,RSTO,RSTR,S0,S1,S2,S3,SF,SH
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,277,15,0.0,0.0,1.0,1.0,0.05,0.06,0.00,255,15,0.06,0.06,0.00,0.00,0.00,0.0,1.00,1.0,21,1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,0,1239,400,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,116,0.45,0.04,0.00,0.00,0.11,0.0,0.02,0.0,18,0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
2,0,222,945,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,25,0.0,0.0,0.0,0.0,1.00,0.00,0.08,57,255,1.00,0.00,0.02,0.03,0.00,0.0,0.00,0.0,21,0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
3,0,235,1380,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,21,0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,256,3,0.0,0.0,1.0,1.0,0.01,0.06,0.00,255,3,0.01,0.08,0.00,0.00,0.00,0.0,1.00,1.0,19,1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False


In [55]:
for x in df_removed_obj.columns:
    print((df_removed_obj[x]).dtype)

int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
float64
float64
float64
float64
float64
float64
float64
int64
int64
float64
float64
float64
float64
float64
float64
float64
float64
int64
int64
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool


In [56]:
target = df_removed_obj.pop('attack')
target

0        1
1        0
2        0
3        0
4        1
        ..
86840    0
86841    0
86842    0
86843    0
86844    0
Name: attack, Length: 86845, dtype: int64

In [57]:
df_removed_obj

,duration,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,rootshell,suattempted,numroot,numfilecreations,numshells,numaccessfiles,numoutboundcmds,ishostlogin,isguestlogin,count,srvcount,serrorrate,srvserrorrate,rerrorrate,srvrerrorrate,samesrvrate,diffsrvrate,srvdiffhostrate,dsthostcount,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,icmp,tcp,udp,IRC,X11,Z39_50,auth,bgp,courier,csnet_ns,ctf,...,iso_tsap,klogin,kshell,ldap,link,login,mtp,name,netbios_dgm,netbios_ns,netbios_ssn,netstat,nnsp,nntp,ntp_u,other,pop_2,pop_3,printer,private,red_i,remote_job,rje,shell,smtp,sql_net,ssh,sunrpc,supdup,systat,telnet,tftp_u,tim_i,time,urh_i,urp_i,uucp,uucp_path,vmnet,whois,OTH,REJ,RSTO,RSTR,S0,S1,S2,S3,SF,SH
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,277,15,0.0,0.0,1.0,1.0,0.05,0.06,0.00,255,15,0.06,0.06,0.00,0.00,0.00,0.00,1.00,1.00,21,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,0,1239,400,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,116,0.45,0.04,0.00,0.00,0.11,0.00,0.02,0.00,18,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
2,0,222,945,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,25,0.0,0.0,0.0,0.0,1.00,0.00,0.08,57,255,1.00,0.00,0.02,0.03,0.00,0.00,0.00,0.00,21,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
3,0,235,1380,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,256,3,0.0,0.0,1.0,1.0,0.01,0.06,0.00,255,3,0.01,0.08,0.00,0.00,0.00,0.00,1.00,1.00,19,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86840,0,216,674,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,106,116,1.00,0.00,0.01,0.03,0.01,0.01,0.00,0.00,21,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,Fal

In [65]:
for x in df_removed_obj.columns:
    df_removed_obj[x] = df_removed_obj[x].astype(float)

In [175]:
for x in df_removed_obj.columns:
    print((df_removed_obj[x]).dtype)

float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64


In [178]:
df_removed_obj.shape

(86845, 116)

In [179]:
df_removed_obj.drop('tftp_u',axis = 1,inplace=True,)

KeyError: "['tftp_u'] not found in axis"

In [180]:
t = tf.convert_to_tensor(df_removed_obj)

In [181]:
t.shape

TensorShape([86845, 116])

In [182]:
normalizer = tf.keras.layers.Normalization(axis=-1)

In [183]:
normalizer.adapt(t)

In [184]:
model = Sequential(
    [
        normalizer,
        Dense(16, activation='relu', name = 'layer1'),
        Dense(5, activation='relu', name = 'layer2'),
        Dense(3, activation = 'relu', name = 'layer3'),
        Dense(1, activation = 'sigmoid',kernel_regularizer = regularizers.L1L2(l1=1e-5, l2=1e-4), name = 'layer4')
     ]
)

In [185]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ normalization_2 (Normalization)      │ (86845, 116)                │             233 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer1 (Dense)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer2 (Dense)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer3 (Dense)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer4 (Dense)                       │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 233 (936.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 233 (936.00 B)

In [186]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

In [187]:
model.fit(t,target,epochs = 50)

Epoch 1/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0217
Epoch 2/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 8.0211e-04
Epoch 3/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 1.7495e-04
Epoch 4/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 2.1893e-04
Epoch 5/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 4.2696e-04
Epoch 6/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - loss: 9.0219e-06
Epoch 7/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 1.6986e-04
Epoch 8/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1.3342e-05
Epoch 9/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - loss: 1.1838e-04
Epoch 10/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 1.6933e-04
Epoch 11/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 4.3219e-05
Epoch 12/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 3.2255e-05
Epoch 13/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 3.8400e-06
Epoch 14/50
2714/2714 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/s

In [188]:
test = pd.read_csv('Test_Data.csv')

In [189]:
test.head()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,rootshell,suattempted,numroot,numfilecreations,numshells,numaccessfiles,numoutboundcmds,ishostlogin,isguestlogin,count,srvcount,serrorrate,srvserrorrate,rerrorrate,srvrerrorrate,samesrvrate,diffsrvrate,srvdiffhostrate,dsthostcount,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag
0,0,tcp,mtp,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,249,7,0.0,0.0,1.0,1.0,0.03,0.07,0.0,255,7,0.03,0.08,0.0,0.0,0.0,0.0,1.0,1.0,20
1,0,tcp,http,SF,199,1721,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,10,0.0,0.0,0.0,0.0,1.00,0.00,0.2,255,255,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,21
2,0,tcp,discard,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,6,1.0,1.0,0.0,0.0,0.38,0.38,0.0,255,14,0.05,0.09,0.0,0.0,1.0,1.0,0.0,0.0,18
3,0,tcp,telnet,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,297,10,1.0,1.0,0.0,0.0,0.03,0.05,0.0,255,2,0.01,0.09,0.0,0.0,1.0,1.0,0.0,0.0,18
4,0,tcp,exec,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,104,19,1.0,1.0,0.0,0.0,0.18,0.06,0.0,255,16,0.06,0.06,0.0,0.0,1.0,1.0,0.0,0.0,20


In [190]:
test2 = test
l = []
for x in test.columns:
    if test[x].dtype==object:
        l.append(x)
        test2 = pd.concat([test2, pd.get_dummies(test[x])],axis = 'columns')
test2.drop(l,axis = 1,inplace = True)
test2.head()

,duration,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,rootshell,suattempted,numroot,numfilecreations,numshells,numaccessfiles,numoutboundcmds,ishostlogin,isguestlogin,count,srvcount,serrorrate,srvserrorrate,rerrorrate,srvrerrorrate,samesrvrate,diffsrvrate,srvdiffhostrate,dsthostcount,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,icmp,tcp,udp,IRC,X11,Z39_50,auth,bgp,courier,csnet_ns,ctf,...,imap4,iso_tsap,klogin,kshell,ldap,link,login,mtp,name,netbios_dgm,netbios_ns,netbios_ssn,netstat,nnsp,nntp,ntp_u,other,pop_2,pop_3,printer,private,red_i,remote_job,rje,shell,smtp,sql_net,ssh,sunrpc,supdup,systat,telnet,tim_i,time,urh_i,urp_i,uucp,uucp_path,vmnet,whois,OTH,REJ,RSTO,RSTR,S0,S1,S2,S3,SF,SH
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,249,7,0.0,0.0,1.0,1.0,0.03,0.07,0.0,255,7,0.03,0.08,0.0,0.0,0.0,0.0,1.0,1.0,20,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,0,199,1721,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,10,0.0,0.0,0.0,0.0,1.00,0.00,0.2,255,255,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,21,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,6,1.0,1.0,0.0,0.0,0.38,0.38,0.0,255,14,0.05,0.09,0.0,0.0,1.0,1.0,0.0,0.0,18,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,297,10,1.0,1.0,0.0,0.0,0.03,0.05,0.0,255,2,0.01,0.09,0.0,0.0,1.0,1.0,0.0,0.0,18,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,104,19,1.0,1.0,0.0,0.0,0.18,0.06,0.0,255,16,0.06,0.06,0.0,0.0,1.0,1.0,0.0,0.0,20,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False


In [191]:
for x in test2.columns:
    test2[x] = test2[x].astype(float)

In [192]:
test2.shape

(21712, 116)

In [193]:
# test2 = pd.concat([test2, pd.Series(np.zeros(test2.shape[0])).rename('tftp_u')],axis='columns')

In [194]:
# test2.head()

In [195]:
test3 = tf.convert_to_tensor(test2)

In [196]:
test3

<tf.Tensor: shape=(21712, 116), dtype=float64, numpy=
array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 1.990e+02, 1.721e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 2.900e+01, 0.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])>

In [197]:
set1 = set(df_removed_obj.columns)

In [198]:
set2 = set(test2.columns)

In [199]:
set1.difference(set2)

set()

In [200]:
set2

{'IRC',
 'OTH',
 'REJ',
 'RSTO',
 'RSTR',
 'S0',
 'S1',
 'S2',
 'S3',
 'SF',
 'SH',
 'X11',
 'Z39_50',
 'auth',
 'bgp',
 'count',
 'courier',
 'csnet_ns',
 'ctf',
 'daytime',
 'diffsrvrate',
 'discard',
 'domain',
 'domain_u',
 'dstbytes',
 'dsthostcount',
 'dsthostdiffsrvrate',
 'dsthostrerrorrate',
 'dsthostsamesrcportrate',
 'dsthostsamesrvrate',
 'dsthostserrorrate',
 'dsthostsrvcount',
 'dsthostsrvdiffhostrate',
 'dsthostsrvrerrorrate',
 'dsthostsrvserrorrate',
 'duration',
 'echo',
 'eco_i',
 'ecr_i',
 'efs',
 'exec',
 'finger',
 'ftp',
 'ftp_data',
 'gopher',
 'hostnames',
 'hot',
 'http',
 'http_443',
 'icmp',
 'imap4',
 'isguestlogin',
 'ishostlogin',
 'iso_tsap',
 'klogin',
 'kshell',
 'land',
 'lastflag',
 'ldap',
 'link',
 'loggedin',
 'login',
 'mtp',
 'name',
 'netbios_dgm',
 'netbios_ns',
 'netbios_ssn',
 'netstat',
 'nnsp',
 'nntp',
 'ntp_u',
 'numaccessfiles',
 'numcompromised',
 'numfailedlogins',
 'numfilecreations',
 'numoutboundcmds',
 'numroot',
 'numshells',
 'ot

In [201]:
set2.difference(set1)

set()

In [203]:
p = model.predict(test3)

679/679 ━━━━━━━━━━━━━━━━━━━━ 1s 831us/step


In [220]:
q = np.array([1 if x>0.5 else 0 for x in p])

In [222]:
q.

dtype('int32')

In [210]:
q.shape

(21712, 1)

In [223]:
np.savetxt("s3.csv",q,delimiter=',')

In [224]:
r = pd.Series(q)

In [228]:
r.rename('attack',inplace = True)

0        1
1        0
2        1
3        1
4        1
        ..
21707    0
21708    0
21709    1
21710    0
21711    1
Name: attack, Length: 21712, dtype: int32

In [229]:
print(r)

0        1
1        0
2        1
3        1
4        1
        ..
21707    0
21708    0
21709    1
21710    0
21711    1
Name: attack, Length: 21712, dtype: int32


In [232]:
r.to_csv('s4.csv',header = True, index = False)